In [1]:
import pandas as pd

df = pd.read_csv("test.csv")

In [2]:
df.head()

,id,latitude,longitude,day_of_year,day_of_week,hour,month
0,0,-42.343,147.372,31,4,15,1
1,1,37.053,127.406,28,1,12,1
2,2,49.105,-123.189,14,1,2,1
3,3,42.014,12.774,10,4,3,1
4,4,47.778,13.002,31,4,11,1


In [3]:
import openmeteo_requests
import pandas as pd
import requests_cache
from retry_requests import retry
from datetime import datetime, timedelta
from tqdm import tqdm
import os

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after=-1)
retry_session = retry(cache_session, retries=5, backoff_factor=0.2)
openmeteo = openmeteo_requests.Client(session=retry_session)

# Weather outputs klasörü oluşturulmamışsa oluştur
os.makedirs("test_weather_outputs", exist_ok=True)

# ID'ye göre döngü
for _, row in tqdm(df.iterrows(), total=len(df)):
    id_ = row["id"]
    lat = row["latitude"]
    lon = row["longitude"]
    day_of_year = int(row["day_of_year"])

    output_path = f"test_weather_outputs/id_{id_}.csv"
    if os.path.exists(output_path):
        print(f"[ID: {id_}] Zaten mevcut, atlanıyor.")
        continue

    # Tarih hesapla
    base_date = datetime(year=2024, month=1, day=1)
    original_date = base_date + timedelta(days=day_of_year - 1)
    start_date = (original_date - timedelta(days=15)).strftime("%Y-%m-%d")
    end_date = (original_date + timedelta(days=15)).strftime("%Y-%m-%d")

    # API parametreleri
    params = {
        "latitude": lat,
        "longitude": lon,
        "start_date": start_date,
        "end_date": end_date,
        "hourly": [
            "temperature_2m", "wind_speed_10m", "wind_speed_100m",
            "wind_direction_10m", "wind_direction_100m", "precipitation",
            "relative_humidity_2m", "surface_pressure", "cloud_cover",
            "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high"
        ],
        "timezone": "auto"
    }

    try:
        responses = openmeteo.weather_api("https://archive-api.open-meteo.com/v1/archive", params=params)
        response = responses[0]
        hourly = response.Hourly()

        hourly_data = {
            "date": pd.date_range(
                start=pd.to_datetime(hourly.Time(), unit="s", utc=True),
                end=pd.to_datetime(hourly.TimeEnd(), unit="s", utc=True),
                freq=pd.Timedelta(seconds=hourly.Interval()),
                inclusive="left"
            ),
            "temperature_2m": hourly.Variables(0).ValuesAsNumpy(),
            "wind_speed_10m": hourly.Variables(1).ValuesAsNumpy(),
            "wind_speed_100m": hourly.Variables(2).ValuesAsNumpy(),
            "wind_direction_10m": hourly.Variables(3).ValuesAsNumpy(),
            "wind_direction_100m": hourly.Variables(4).ValuesAsNumpy(),
            "precipitation": hourly.Variables(5).ValuesAsNumpy(),
            "relative_humidity_2m": hourly.Variables(6).ValuesAsNumpy(),
            "surface_pressure": hourly.Variables(7).ValuesAsNumpy(),
            "cloud_cover": hourly.Variables(8).ValuesAsNumpy(),
            "cloud_cover_low": hourly.Variables(9).ValuesAsNumpy(),
            "cloud_cover_mid": hourly.Variables(10).ValuesAsNumpy(),
            "cloud_cover_high": hourly.Variables(11).ValuesAsNumpy(),
        }

        hourly_df = pd.DataFrame(hourly_data)
        hourly_df.to_csv(output_path, index=False)

    except Exception as e:
        print(f"[ID: {id_}] Veri çekilirken hata oluştu: {e}")

100%|██████████| 2739/2739 [00:00<00:00, 28718.34it/s]

[ID: 0.0] Zaten mevcut, atlanıyor.
[ID: 1.0] Zaten mevcut, atlanıyor.
[ID: 2.0] Zaten mevcut, atlanıyor.
[ID: 3.0] Zaten mevcut, atlanıyor.
[ID: 4.0] Zaten mevcut, atlanıyor.
[ID: 5.0] Zaten mevcut, atlanıyor.
[ID: 6.0] Zaten mevcut, atlanıyor.
[ID: 7.0] Zaten mevcut, atlanıyor.
[ID: 8.0] Zaten mevcut, atlanıyor.
[ID: 9.0] Zaten mevcut, atlanıyor.
[ID: 10.0] Zaten mevcut, atlanıyor.
[ID: 11.0] Zaten mevcut, atlanıyor.
[ID: 12.0] Zaten mevcut, atlanıyor.
[ID: 13.0] Zaten mevcut, atlanıyor.
[ID: 14.0] Zaten mevcut, atlanıyor.
[ID: 15.0] Zaten mevcut, atlanıyor.
[ID: 16.0] Zaten mevcut, atlanıyor.
[ID: 17.0] Zaten mevcut, atlanıyor.
[ID: 18.0] Zaten mevcut, atlanıyor.
[ID: 19.0] Zaten mevcut, atlanıyor.
[ID: 20.0] Zaten mevcut, atlanıyor.
[ID: 21.0] Zaten mevcut, atlanıyor.
[ID: 22.0] Zaten mevcut, atlanıyor.
[ID: 23.0] Zaten mevcut, atlanıyor.
[ID: 24.0] Zaten mevcut, atlanıyor.
[ID: 25.0] Zaten mevcut, atlanıyor.
[ID: 26.0] Zaten mevcut, atlanıyor.
[ID: 27.0] Zaten mevcut, atlanıyor.
[I

In [4]:
df["latitude"].isna().sum()

np.int64(0)

In [5]:
df["longitude"].isna().sum()

np.int64(0)

In [6]:
df.to_csv("airpol_test.csv", index=False)

1524
